<a href="https://colab.research.google.com/github/Aaliya201/CAD_branch_predictor/blob/main/CAD_gshare_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving trace_01.txt to trace_01.txt


In [3]:
file=open('trace_01.txt','r')
global_reg=0
index=0

history=[0] * 1024                                #This array is used to simulate the behaviour of pattern history table

mispredict=0                                 #Used for the misprediction counter
line_counter=0                                     #Counts the number of lines present in the trace file
bitwidth=0                                     #Used to take the bitwidth for the saturating counter
centerval=0                                 #Calculates the center value for a particular bit counter size(eg: 1 for a 2 bit counter)


bitwidth=2 #Enter the counter size as 2,4,6,8 or 10 (bits)
centerval=(((2**bitwidth)-1)/2) 

for line in file:
    line=line.rstrip()                         
    words=line.split()        #splits the string into a list                      
    index=int(words[0],0)%1024
  #  print (words[0])
    
    
    if words[1]=='0':
        if history[global_reg]<=centerval and history[global_reg]>0:
            history[global_reg]=history[global_reg]-1
          #  print(history[global_reg])
           
        elif history[global_reg]>centerval:
            history[global_reg]=history[global_reg]-1
            mispredict=mispredict+1

        global_reg=((global_reg<<1) & 0x000003ff)   # the branch outcome is shifted into the most significant bit of the global history register      
        global_reg=global_reg^index         # global register is XORed with the program counter which happens to be index in this case
        
                        
    elif words[1]=='1':
        if history[global_reg]<=centerval:
            history[global_reg]=history[global_reg]+1
            mispredict=mispredict+1
        elif history[global_reg]>centerval and history[global_reg]<((2**bitwidth)-1): # 2^bitwidth which is 2 here 
            history[global_reg]=history[global_reg]+1

        global_reg=(((global_reg<<1) | 0x01) & 0x000003ff) 
        global_reg=global_reg^index       

    line_counter=line_counter+1


pred=format(100-(100*(mispredict/(line_counter*1.0))),'0.02f')

print ("The prediction rate for a ",bitwidth," bit counter is:", pred, "%")
print ("The misprediction rate for a ",bitwidth," bit counter is:", (100-float(pred)), "%")

The prediction rate for a  2  bit counter is: 98.81 %
The misprediction rate for a  2  bit counter is: 1.1899999999999977 %
